## Client Requirements
Your report (in the discussion section) must include the following:

Some discussion of the features that are most important for predicting a cancelation - we do not need discussion of specific coefficient values but direction of the effect should be clear (e.g. the earlier a booking is made the more likely it is to be canceled).

A validated assessment of your model's performance, but this must be specifically discussed in the context of bookings and running a hotel. It is not sufficient to report summary statistics like the accuracy or AUC - you must address the perfomance in terms of potential gains and losses for the hotel (e.g. think about what happens if your model predicts a cancelation that does not actually occur and a room ends up being double booked or vice versa). Explain why you think your particular model would or would not be economically viable.

# Planning Markdown
Where to write what we are going to do & how - so that others can puck this up and we can code togteher

Feature engineering 
To Do:
* Add FX date for country currency for the arrival & booking date & see the %change between these dates. 
* Split data by the resort type. These will have a difference clientel, so will be different models. 

Model fitting 
* Still to decide on a model, however this will come out of anaysis from splitting date into resort/city ti see what is appropriate. 
* Cut down features for each dataset & decide on the best fuitable model by what the remaing features are 

# Machine Learning in Python - Project 1

Due Friday, April 9th by 5 pm UK local time.

*include contributors names here*

## 0. Setup

In [1]:
# Install required packages
!pip install -q -r requirements.txt

In [2]:
# Add any additional libraries or submodules below

# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd
import numpy as np
import geopy.distance as gpy
import plotly.express as px
from datetime import datetime
import pycountry as pyc
import ccy

#Web Scraping Requirement
import datapackage

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules
import sklearn

In [3]:
# Load data
d = pd.read_csv("hotel.csv")

## 1. Introduction

*This section should include a brief introduction to the task and the data (assume this is a report you are delivering to a client). If you use any additional data sources, you should introduce them here and discuss why they were included.*

*Briefly outline the approaches being used and the conclusions that you are able to draw.*

## 2. Exploratory Data Analysis and Feature Engineering

### Data Cleansing

Join useful country data to main "d"

The data has 4 columns with "nan", these columns were: children , country , agent , company - resulting in 87138 rows of incomplete data entries. 
* nan children were replaced with zeros - with the assumption that this data collection omitted filling this field in for no children. 
* Agent & Company have many nan values and was cosidered to be dropped from the dataset. From brief analysis, there are correlations between the agent/company and the country of the booking. Given that there is the 'distribution_channel' column, this will identify whether an agent/company was used or otherwise. nan values will be replaced with zeros.
* For entries with a 'nan' country, these have also assumed to have been defaulted to PRT (Portugal), given that the hotels are in Portugal and make up 32% of all bookings; Of ~190,00 row, this is a fair rough assumption given there are only 415 'nan' countries. 

There are ~32,500 duplicate rows of data. In the interest of meeting the clients requirements, we are investigating what will indicate cancellations, and not the overall booking. It would be fair to assume these were all "double bookings" are multiple groups of friends booking for the same holiday in different transactions, or for a popular event - for this reason we will not be dropping these duplicates. The other reason for not dropping the duplicates is because 90% of the duplicates are reservations for the City hotel, which would remove potentially useful data. 

In [4]:
#replace company/agent values with 0 to represent no agent/company was used when booking. 
d['company'].fillna('0',inplace=True)
d['agent'].fillna('0',inplace=True)
#nan children will be zero children 
d['children'].fillna('0',inplace=True)
#Replace nan country with PRT (Portugal)
d['country'].fillna('PRT',inplace=True)
#Create direct booking flag
d['direct_booking'] = np.where(((d.distribution_channel == 'Direct')), 1, 0)

The Date of the booking & the Date of when the Hotel stay is added to the  DataFrame to allow for further processing. 

In [5]:
#format arrival date
d['month'] = pd.to_datetime(d.arrival_date_month, format='%B').dt.month
d['day'] = pd.to_datetime(d.arrival_date_day_of_month, format='%d').dt.day
d['Year'] = pd.to_datetime(d.arrival_date_year, format='%Y').dt.year
d['arrival_date']  = pd.to_datetime(d[['Year','month','day']], format = '%Y%m%d')#.dt.date
d['booking_date'] = d['arrival_date'] - pd.to_timedelta(d['lead_time'], unit='d')
d.drop(['day','Year','month'],axis =1, inplace=True )
min_date = d['arrival_date'].min()
max_date = d['arrival_date'].max()
min_booking_date = d['booking_date'].min()


### Additional Data sources & processing

The below additional datasets have been added to enrich the study:

|Dataframe                       |description |
|:------------------------------|:-----------|
|cur_code                          | Country Names & associated Currencyfor that country |
|countries_name                      | ISO 3155 Country Names & 3 character currency code to join with hotel.csv |
|coords             | central coordinates of each country & ISO 3155 Country code|
|fx_rates             | Daily Global Currecny Exchange rates from 1995 - 2018 (in USD)  |
|comp_countries            | Comprehensive country code information, including ISO 3166 codes, ITU dialing codes, ISO 4217 currency codes, |

In [6]:
cur_code = pd.read_csv("curr_codes.csv")
countries_name = pd.read_csv("ISO 3155.csv")
coords = pd.read_csv("countries_coords.csv")
fx_rates = pd.read_csv("/work/currency_exchange_rates_02-01-1995_-_02-05-2018.csv")

data_url = 'https://datahub.io/core/country-codes/datapackage.json'
# to load Data Package into storage
package = datapackage.Package(data_url)
# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        comp_countries = pd.read_csv(resource.descriptor['path'])

|Dataframe                       |Column |Dataframe                       |Column |
|:------------------------------|:-----------|:-----------|:-----------|
|fx_rates                   | currency_name| cur_code| currency_name |
|d                   | arrival_date, currency_code| fx_rates| Date, currency_code |
|cur_code                          |  ||  |
|countries_name  |   ||  |
|coords             | ||  |
|fx_rates           |  ||  |
|comp_countries |  ||  |

Additional Countey data can be added to the main Dataset. This includes the following:
* Continent
* Country Capital (probably useless)
* whether a country is independant
* The distance from the Hotels which the visting country is from the Portugal. 



In [7]:
# Join comp_countries data to d (main)
comp_countries = comp_countries.loc[:, ['ISO3166-1-Alpha-3',
                                'ISO3166-1-Alpha-2',
                                'ISO4217-currency_country_name',
                                'ISO4217-currency_alphabetic_code',
                                'Region Name',
                                'is_independent', 
                                'Continent']]
d = pd.merge(d,comp_countries,left_on='country', right_on='ISO3166-1-Alpha-3', how = 'left')
d.rename(columns = {'ISO4217-currency_alphabetic_code' : 'Currency_code'}, inplace = True)

#Reform Co-Ordinates into list within DF
coords['co_ords'] =  coords[['latitude', 'longitude']].values.tolist()
coords = coords.dropna()

# Set Portugal as basis
portugal = coords['co_ords'].loc[coords["name"] == 'Portugal'].values.tolist()

#Compute the distance in KM from all countries to Portugal
coords['distance(km)'] = coords.apply(lambda coords: gpy.great_circle(portugal,
                                                             coords['co_ords']).km,
                                                            axis = 1).round(decimals=2)
# Join to main Dataframe
coords_cut = coords[['country','distance(km)']]
coords_cut.columns = ['ISO3166-1-Alpha-2','distance(km)']
d = pd.merge(d,coords_cut ,on='ISO3166-1-Alpha-2', how = 'left')


#Rename the columns within FX Rates Column
#Select Columns from cur_code
cur_code = cur_code.loc[:, ['country Entity','Currency','AlphabeticCode']]

# Get all FX Rate currency codes (don't include Date field)
fx_codes = pd.DataFrame(fx_rates.columns[1:], )

#Test the results of the merge codes which match well currently
cur_mapping_test = pd.merge(fx_codes,cur_code,left_on=0, right_on='Currency', how = 'left') 

#check for non matching
#sorted(cur_mapping_test[0][cur_mapping_test['Currency'].isna()])
# Create Update Dict:
null_curr_map = sorted(cur_mapping_test[0][cur_mapping_test['Currency'].isna()])
new_curr_map = ('Bahraini Dinar',
                'Pula',
                'Yuan Renminbi',
                'Forint',
                'Iceland Krona',
                'Rupiah',
                'New Israeli Sheqel',
                'Yen',
                'Tenge',
                'Won',
                'Mauritius Rupee',
                'Pakistan Rupee',
                'Zloty',
                'Qatari Rial',
                'Saudi Riyal',
                'Rand',
                'Baht',
                'Trinidad and Tobago Dollar',
                'UAE Dirham',
                'Pound Sterling',
                'US Dollar',
)
# Create Dataframe to update old values within FX Code to (i) join to Main and (ii) update fxrate columns
fx_rename = dict(zip(null_curr_map, new_curr_map))

# Update FX Rates - ready for updateing fx rates
fx_rates.rename(columns = fx_rename, inplace = True)

# You need this mapping for when the FX Rate data is processed & pivoted. 
cur_mapping =  pd.merge(fx_codes,cur_code,left_on=0, right_on='Currency', how = 'left') 

Exchange rates for all Common currencies have been sourced from 1995 - 2018. This data can be processed and cur to investigate the status of the strength of the guests currency from at the time of booking to when the holiday is close by. This requires some preprocessing of the data for it to be added to the main Dataset. 

In [8]:

#Reformat Date column
fx_rates['Date'] = pd.to_datetime(fx_rates['Date'], format='%Y-%m-%d')
#fx_rates['Date']  = pd.to_datetime(fx_rates['Date']).dt.date

#Reform into Euro conversion rates  
fx_rates.iloc[:,1:] = fx_rates.iloc[:,1:].div(fx_rates.Euro, axis=0)

#Cut Dates so that theres only the date from the earliest booking to the last booking
fx_rates = fx_rates[fx_rates['Date'].between(min_booking_date,max_date)]

# Create rows full of "nan" for the missing dates within range 
fx_rates = pd.merge(pd.DataFrame({"Date" : pd.date_range(min_booking_date, max_date, freq='D')}),fx_rates ,left_on='Date', right_on='Date', how = 'outer')

# Interpolate to remove missing true null values  and newly created values for dates
fx_rates = fx_rates.interpolate()

#Pivot Data vertically 
fx_rates = fx_rates.melt(id_vars = ['Date'],var_name='Currency', value_name='FX_Rates')

# Join to get currency code
d.rename(columns = {'ISO4217-currency_alphabetic_code' : 'Currency_code'}, inplace = True)
cur_code.rename(columns = {'AlphabeticCode' : 'Currency_code'}, inplace = True)
fx_rates = pd.merge(fx_rates, cur_code,on='Currency', how = 'left')
fx_rates = fx_rates[['Date','Currency','Currency_code','FX_Rates']]

fx_rates = fx_rates.drop_duplicates()

# get FX rate as at arrival date
d = pd.merge(d, fx_rates,left_on=['Currency_code','arrival_date'], right_on = ['Currency_code','Date'] ,how = 'left' )
d.rename(columns = {'FX_Rates' : 'FX_Rates_on_arrival'}, inplace = True)
# get FX rate as at booking date
d = pd.merge(d, fx_rates,left_on=['Currency_code','booking_date' ], right_on = ['Currency_code','Date'],how = 'left' )
d.rename(columns = {'FX_Rates' : 'FX_Rates_on_booking'}, inplace = True)
# Drop useless columns from merges 
d.drop(['Date_x','Date_y','Currency_x','Currency_y'],axis =1, inplace=True )
# Calculate the % Loss between Booking & Arrival Dates
d['FX_Delta'] = np.where(((d.Currency_code =='EUR')|(d['FX_Rates_on_arrival'] - d['FX_Rates_on_booking'] ==0)),0, (d['FX_Rates_on_arrival'] - d['FX_Rates_on_booking'])/ d['FX_Rates_on_booking'])


In [25]:
d[d.columns[d.isna().any()]]

,ISO3166-1-Alpha-3,ISO3166-1-Alpha-2,ISO4217-currency_country_name,Currency_code,Region Name,is_independent,Continent,distance(km),FX_Rates_on_arrival,FX_Rates_on_booking,FX_Delta
0,PRT,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,0.000000
1,PRT,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,0.000000
2,GBR,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.405155,0.003961
3,GBR,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.394511,0.011624
4,GBR,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.395780,0.010704
...,...,...,...,...,...,...,...,...,...,...,...
119385,BEL,BE,BELGIUM,EUR,Europe,Yes,EU,1583.63,1.000000,1.000000,0.000000
119386,FRA,FR,FRANCE,EUR,Europe,Yes,EU,1139.01,1.000000,1.000000,0.000000
119387,DEU,DE,GERMANY,EUR,Europe,Yes,EU,1951.04,1.000000,1.000000,0.000000
119388,GBR,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.088879,1.179501,-0.076831


In [26]:
d[d['ISO3166-1-Alpha-3'].isna()]

,is_canceled,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,ISO3166-1-Alpha-2,ISO4217-currency_country_name,Currency_code,Region Name,is_independent,Continent,distance(km),FX_Rates_on_arrival,FX_Rates_on_booking,FX_Delta
143,0,Resort Hotel,109,2015,July,28,5,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,0,Resort Hotel,94,2015,July,28,6,1,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,0,Resort Hotel,53,2015,July,29,12,1,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,0,Resort Hotel,100,2015,July,29,12,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,0,Resort Hotel,3,2015,July,29,18,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119056,0,City Hotel,17,2017,August,35,27,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119220,0,City Hotel,167,2017,August,35,30,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119227,0,City Hotel,167,2017,August,35,30,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119302,0,City Hotel,217,2017,August,35,28,1,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
np.max((d['FX_Rates_on_arrival'] - d['FX_Rates_on_booking'])/d['FX_Rates_on_booking'])


0.3761684696968484

In [11]:
#s+===s
regional_booking = d.groupby(['hotel','ISO4217-currency_country_name',]).agg({'is_canceled':'count'}).reset_index()


In [12]:
fig = px.sunburst(d.groupby(['hotel','is_canceled',]).agg({'ISO4217-currency_country_name':'count'}).reset_index(),
                    path = ['hotel', 'is_canceled',],
                    values = 'ISO4217-currency_country_name',
                    title = 'Regional Distribution between Hotels ')

fig.show()

In [13]:
fig = px.sunburst(d.groupby(['hotel','ISO4217-currency_country_name',]).agg({'is_canceled':'count'}).reset_index(),
                    path = ['hotel', 'ISO4217-currency_country_name',],
                    values = 'is_canceled',
                    title = 'Regional Distribution between Hotels ')

fig.show()

In [14]:

d
#d['FX_Rates_on_booking'] = np.where(((d.Currency_code == 'EUR')), 1.0 ,'Testing what is going on')

#d['FX_Rates_on_booking'] = np.where(((d.Currency_code == 'EUR')&(d.FX_Rates_on_booking.isnull() )), 1.0 ,'Testing what is going on')

,is_canceled,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,ISO3166-1-Alpha-2,ISO4217-currency_country_name,Currency_code,Region Name,is_independent,Continent,distance(km),FX_Rates_on_arrival,FX_Rates_on_booking,FX_Delta
0,0,Resort Hotel,342,2015,July,27,1,0,0,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
1,0,Resort Hotel,737,2015,July,27,1,0,0,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
2,0,Resort Hotel,7,2015,July,27,1,0,1,1,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.405155,0.003961
3,0,Resort Hotel,13,2015,July,27,1,0,1,1,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.394511,0.011624
4,0,Resort Hotel,14,2015,July,27,1,0,2,2,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.395780,0.010704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,City Hotel,23,2017,August,35,30,2,5,2,...,BE,BELGIUM,EUR,Europe,Yes,EU,1583.63,1.000000,1.000000,1.000000
119386,0,City Hotel,102,2017,August,35,31,2,5,3,...,FR,FRANCE,EUR,Europe,Yes,EU,1139.01,1.000000,1.000000,1.000000
119387,0,City Hotel,34,2017,August,35,31,2,5,2,...,DE,GERMANY,EUR,Europe,Yes,EU,1951.04,1.000000,1.000000,1.000000
119388,0,City Hotel,109,2017,August,35,31,2,5,2,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.088879,1.179501,-0.076831


In [15]:
#d[((d['FX_Rates_on_booking'] == 'Testing what is going on') & (d['Currency_code']  == 'EUR') )]
fx_rates[(fx_rates['Date'] == '2017-05-14T00:00:00.000000')]


/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:822: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:850: RuntimeWarning:

invalid value encountered in multiply



,Date,Currency,Currency_code,FX_Rates
1420,2017-05-14,Algerian Dinar,DZD,99.884941
12890,2017-05-14,Australian Dollar,AUD,0.675908
15190,2017-05-14,Bahraini Dinar,BHD,0.343699
16720,2017-05-14,Bolivar Fuerte,VEF,9.118072
18250,2017-05-14,Pula,BWP,0.087995
19780,2017-05-14,Brazilian Real,BRL,2.867891
21310,2017-05-14,Brunei Dollar,BND,1.284463
22840,2017-05-14,Canadian Dollar,CAD,1.249151
24370,2017-05-14,Chilean Peso,CLP,613.542843
25900,2017-05-14,Yuan Renminbi,CNY,6.307065


In [16]:
fx_rates['Date'] = pd.to_datetime(fx_rates['Date'], format='%Y-%m-%d')
fx_rates['Date'].dtypes
fx_rates = fx_rates.drop_duplicates()

In [17]:
fx_rates[(fx_rates['Date'] == '2015-07-01')& (fx_rates['Currency_code'] == 'EUR')  ] #& (fx_rates['Currency_code'] == 'EUR') 

/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:822: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:850: RuntimeWarning:

invalid value encountered in multiply



,Date,Currency,Currency_code,FX_Rates
60192,2015-07-01,Euro,EUR,1.0


In [18]:
d.head(30)


/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:822: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:850: RuntimeWarning:

invalid value encountered in multiply



,is_canceled,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,ISO3166-1-Alpha-2,ISO4217-currency_country_name,Currency_code,Region Name,is_independent,Continent,distance(km),FX_Rates_on_arrival,FX_Rates_on_booking,FX_Delta
0,0,Resort Hotel,342,2015,July,27,1,0,0,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
1,0,Resort Hotel,737,2015,July,27,1,0,0,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
2,0,Resort Hotel,7,2015,July,27,1,0,1,1,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.405155,0.003961
3,0,Resort Hotel,13,2015,July,27,1,0,1,1,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.394511,0.011624
4,0,Resort Hotel,14,2015,July,27,1,0,2,2,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.395780,0.010704
5,0,Resort Hotel,14,2015,July,27,1,0,2,2,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.395780,0.010704
6,0,Resort Hotel,0,2015,July,27,1,0,2,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
7,0,Resort Hotel,9,2015,July,27,1,0,2,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
8,1,Resort Hotel,85,2015,July,27,1,0,3,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
9,1,Resort Hotel,75,2015,July,27,1,0,3,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000


In [19]:
#d = pd.merge(d, fx_rates,left_on=['Currency_code','booking_date'], right_on = ['Currency_code','Date'])
#d.rename(columns = {'FX_Rates' : 'FX_Rates_on_booking'}, inplace = True)
d


,is_canceled,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,ISO3166-1-Alpha-2,ISO4217-currency_country_name,Currency_code,Region Name,is_independent,Continent,distance(km),FX_Rates_on_arrival,FX_Rates_on_booking,FX_Delta
0,0,Resort Hotel,342,2015,July,27,1,0,0,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
1,0,Resort Hotel,737,2015,July,27,1,0,0,2,...,PT,PORTUGAL,EUR,Europe,Yes,EU,0.00,1.000000,1.000000,1.000000
2,0,Resort Hotel,7,2015,July,27,1,0,1,1,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.405155,0.003961
3,0,Resort Hotel,13,2015,July,27,1,0,1,1,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.394511,0.011624
4,0,Resort Hotel,14,2015,July,27,1,0,2,2,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.410721,1.395780,0.010704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,City Hotel,23,2017,August,35,30,2,5,2,...,BE,BELGIUM,EUR,Europe,Yes,EU,1583.63,1.000000,1.000000,1.000000
119386,0,City Hotel,102,2017,August,35,31,2,5,3,...,FR,FRANCE,EUR,Europe,Yes,EU,1139.01,1.000000,1.000000,1.000000
119387,0,City Hotel,34,2017,August,35,31,2,5,2,...,DE,GERMANY,EUR,Europe,Yes,EU,1951.04,1.000000,1.000000,1.000000
119388,0,City Hotel,109,2017,August,35,31,2,5,2,...,GB,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,GBP,Europe,Yes,EU,1811.82,1.088879,1.179501,-0.076831


In [20]:
#-- DO NOT TRY AND MAP A CURRANCY STRAIGHT TO THE COUNTRY, AS YOU WILL GET 1:MANY MAPPPING!
cut_cur_code = cur_code[['country Entity','Currency','AlphabeticCode']]
cut_cur_code.columns = ['Country','Currency','ISO4217-currency_alphabetic_code']

d= pd.merge(d,cut_cur_code[['Currency','ISO4217-currency_alphabetic_code']],on='ISO4217-currency_alphabetic_code', how = 'left') 
d

KeyError: "['AlphabeticCode'] not in index"

Ensure the FX Rates & the Main "d" dataset have joinable columns - a small amount of data cleaning is required manually. 

In [ ]:
!pip install pycountry==20.7.3

In [ ]:
import pycountry as pyc
pd.DataFrame(pyc.countries)

,0
0,"Country(alpha_2='AW', alpha_3='ABW', name='Aru..."
1,"Country(alpha_2='AF', alpha_3='AFG', name='Afg..."
2,"Country(alpha_2='AO', alpha_3='AGO', name='Ang..."
3,"Country(alpha_2='AI', alpha_3='AIA', name='Ang..."
4,"Country(alpha_2='AX', alpha_3='ALA', name='Åla..."
...,...
244,"Country(alpha_2='WS', alpha_3='WSM', name='Sam..."
245,"Country(alpha_2='YE', alpha_3='YEM', name='Yem..."
246,"Country(alpha_2='ZA', alpha_3='ZAF', name='Sou..."
247,"Country(alpha_2='ZM', alpha_3='ZMB', name='Zam..."


In [ ]:
import ccy
#cur_mapping

#print(ccy.countryccy(d['ISO3166-1-Alpha-2']))

In [ ]:
#d['currency_code'] = ccy.countryccy(d['ISO3166-1-Alpha-2'])
d['currency_code'] = d.apply(lambda d: ccy.countryccy(d['ISO3166-1-Alpha-2']),
                                                            axis = 1)


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
from dataprep.clean import clean_country

clean_country(df, 'country Entity', output_format="alpha-3")

*Include a detailed discussion of the data with a particular emphasis on the features of the data that are relevant for the subsequent modeling. Including visualizations of the data is strongly encouraged - all code and plots must also be described in the write up. Think carefully about whether each plot needs to be included in your final draft - your report should include figures but they should be as focused and impactful as possible.*

*Additionally, this section should also implement and describe any preprocessing / feature engineering of the data. Specifically, this should be any code that you use to generate new columns in the data frame `d`. All of this processing is explicitly meant to occur before we split the data in to training and testing subsets. Processing that will be performed as part of an sklearn pipeline can be mentioned here but should be implemented in the following section.*

*All code and figures should be accompanied by text that provides an overview / context to what is being done or presented.*

From data collected by [Antonio, Almeida and Nunes, 2019](https://www.sciencedirect.com/science/article/pii/S2352340918315191#f0010),  we found that "Data source location	Both hotels are located in Portugal: H1 at the resort region of Algarve and H2 at the city of Lisbon". This can be used with the ISO code and compare to the location of the guests, and how far they are travelling roughly using average co-ordinates. 

'ISO 3155.csv' - Gives ISO 3155 ISO codes and names - https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3

'countries_coords.csv' - Gives location of Countries with ISO 2 letter code
ISO 3155. https://developers.google.com/public-data/docs/canonical/countries_csv


Comprehensive country codes: ISO 3166, ITU, ISO 4217 currency codes and many more: https://datahub.io/core/country-codes 

### Formatting Arrival & Booking Date
Formatting to allow for joining of external data sources

In [ ]:
fx_rates.iloc[:,1:].rolling(window=90, min_periods = 1).mean()
#fx_rates.iloc[:,1:].rolling(5)

,Algerian Dinar,Australian Dollar,Bahrain Dinar,Bolivar Fuerte,Botswana Pula,Brazilian Real,Brunei Dollar,Canadian Dollar,Chilean Peso,Chinese Yuan,...,South African Rand,Sri Lanka Rupee,Swedish Krona,Swiss Franc,Thai Baht,Trinidad And Tobago Dollar,Tunisian Dinar,U.A.E. Dirham,U.K. Pound Sterling,U.S. Dollar
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,0.376,NaN,NaN,0.843000,NaN,1.403500,NaN,NaN,...,3.550000,50.040000,7.458800,1.316800,25.100000,5.875000,NaN,3.6710,1.561700,1.0
2,NaN,0.770400,0.376,NaN,NaN,0.843500,NaN,1.403050,NaN,NaN,...,3.555000,50.055000,7.453400,1.316450,25.115000,5.874400,NaN,3.6710,1.561100,1.0
3,NaN,0.769850,0.376,NaN,NaN,0.843667,NaN,1.402100,NaN,NaN,...,3.553333,50.060000,7.450600,1.313633,25.123333,5.872933,NaN,3.6710,1.561067,1.0
4,NaN,0.769867,0.376,NaN,NaN,0.843250,NaN,1.401625,NaN,NaN,...,3.555000,50.062500,7.450600,1.311225,25.125000,5.874950,NaN,3.6710,1.561925,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5973,114.228647,0.781959,0.376,23177.657382,0.103569,3.282634,1.320564,1.266659,603.654138,6.367382,...,12.039464,154.864606,8.194687,0.955339,31.624012,6.757433,2.441547,3.6725,1.390511,1.0
5974,114.224845,0.781823,0.376,24044.672722,0.103600,3.284352,1.320302,1.266411,603.251149,6.364139,...,12.029338,154.919127,8.197227,0.955373,31.612578,6.757623,2.440849,3.6725,1.390962,1.0
5975,114.219670,0.781704,0.376,24938.406813,0.103630,3.286570,1.320020,1.266349,603.054368,6.360908,...,12.025161,154.974517,8.200304,0.955434,31.596976,6.757401,2.440134,3.6725,1.391357,1.0
5976,114.207343,0.781561,0.376,25266.412495,0.103673,3.286557,1.319708,1.266123,602.943563,6.358207,...,12.016735,155.027912,8.198160,0.955056,31.582866,6.757002,2.439468,3.6725,1.391696,1.0


In [ ]:
#Reformat Date column
fx_rates['Date']  = pd.to_datetime(fx_rates['Date']).dt.date

#Divide all by Euros - All FX Rates are: "Currency" per Euro
fx_rates.iloc[:,1:] = fx_rates.iloc[:,1:].div(fx_rates.Euro, axis=0)

#calculate rolling 90 day average
new_col_names = list('3mth_avg_'+fx_rates.columns[1:])
three_mth_avg = fx_rates.iloc[:,1:].rolling(window=90, min_periods = 1).mean()
three_mth_avg.columns = new_col_names
fx_rates =  pd.concat([fx_rates, three_mth_avg], axis=1)

#Cut Dates so that theres only the date from the earliest booking to the last booking
fx_rates = fx_rates[fx_rates['Date'].between(min_booking_date,max_date)]

In [ ]:
fx_rates

,Date,Algerian Dinar,Australian Dollar,Bahrain Dinar,Bolivar Fuerte,Botswana Pula,Brazilian Real,Brunei Dollar,Canadian Dollar,Chilean Peso,...,3mth_avg_South African Rand,3mth_avg_Sri Lanka Rupee,3mth_avg_Swedish Krona,3mth_avg_Swiss Franc,3mth_avg_Thai Baht,3mth_avg_Trinidad And Tobago Dollar,3mth_avg_Tunisian Dinar,3mth_avg_U.A.E. Dirham,3mth_avg_U.K. Pound Sterling,3mth_avg_U.S. Dollar
4764,2013-06-24,60.358551,0.703194,0.287330,4.802231,0.088644,1.730705,0.975470,0.804830,393.076570,...,7.145895,96.995420,4.984052,0.720928,22.782825,4.909261,1.230295,2.812987,1.170491,0.765932
4765,2013-06-25,60.255444,0.704431,0.286280,4.784681,0.088473,1.714253,0.971372,0.800442,390.893863,...,7.157703,97.034088,4.988535,0.721151,22.797430,4.910033,1.231374,2.813458,1.170647,0.766057
4766,2013-06-26,61.209690,0.711686,0.288698,4.825092,0.088990,1.703394,0.976351,0.803824,390.479115,...,7.170559,97.082158,4.993390,0.721515,22.815356,4.911936,1.232611,2.814266,1.170883,0.766272
4767,2013-06-27,61.180786,0.715009,0.288521,4.822130,0.089319,1.686234,0.973297,0.804174,387.523020,...,7.182678,97.135327,4.998980,0.721948,22.834317,4.913568,1.233905,2.815104,1.171210,0.766495
4768,2013-06-28,61.179817,0.709098,0.287462,4.804434,0.088838,1.670183,0.967278,NaN,385.214067,...,7.193173,97.168465,5.002608,0.722137,22.848322,4.914294,1.235080,2.815370,1.171298,0.766565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5807,2017-08-25,93.068005,0.669461,0.318428,8.447663,0.083079,2.659045,1.152270,1.057927,540.853659,...,11.603584,134.770078,7.518180,0.859178,29.975468,5.958620,2.166343,3.241311,1.139605,0.882556
5808,2017-08-28,92.098197,0.666080,0.315304,8.364780,0.082767,2.637820,1.137275,1.046709,533.844864,...,11.587999,134.691777,7.498833,0.857728,29.926316,5.950820,2.164029,3.237468,1.138373,0.881521
5809,2017-08-29,91.317812,0.657869,0.312085,8.279382,0.082337,2.619273,1.124419,1.038762,523.182271,...,11.569522,134.510297,7.477626,0.856032,29.871275,5.941485,2.161055,3.233097,1.137013,0.880344
5810,2017-08-30,92.958711,0.669772,0.315542,8.371098,0.082830,2.659366,1.136791,1.057821,525.914736,...,11.557653,134.367767,7.459990,0.854734,29.825867,5.934179,2.158536,3.229649,1.135923,0.879416


In [ ]:

sample = fx_rates.head(-500)


new_col_names = sample.columns[:1].tolist() + list('90_avg'+fx_rates.columns[1:])
sample.iloc[:,1:] = sample.iloc[:,1:].rolling(window=90).mean()
sample.columns = new_col_names
sample =  pd.concat([fx_rates.head(500), sample.iloc[:,1:]], axis=1)
sample


#rolling_averages = {}
#for i in range(1,len(fx_rates.columns))
#    name = "90_ave_"+col_name
#    fx_rates[name] = fx_rates.iloc[:,i].rolling(window=90).mean()



/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


,Date,Algerian Dinar,Australian Dollar,Bahrain Dinar,Bolivar Fuerte,Botswana Pula,Brazilian Real,Brunei Dollar,Canadian Dollar,Chilean Peso,...,90_avgSouth African Rand,90_avgSri Lanka Rupee,90_avgSwedish Krona,90_avgSwiss Franc,90_avgThai Baht,90_avgTrinidad And Tobago Dollar,90_avgTunisian Dinar,90_avgU.A.E. Dirham,90_avgU.K. Pound Sterling,90_avgU.S. Dollar
0,1995-1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1995-1-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1995-1-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1995-1-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1995-1-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.987333,NaN,NaN,NaN,3.6725,NaN,1.0
5474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.987029,NaN,NaN,NaN,3.6725,NaN,1.0
5475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.986668,NaN,NaN,NaN,3.6725,NaN,1.0
5476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.986317,NaN,NaN,NaN,3.6725,NaN,1.0


In [ ]:
new_col_names = list('90_avg'+fx_rates.columns[1:])

#### Geographical location
The distance in kilometers was calculated for all countries, based on their central location, to base the distance of each traveller. this will also act as act as a proxy for the cost of travel to the location. 

## 3. Model Fitting and Tuning

*In this section you should detail your choice of model and describe the process used to refine and fit that model. You are strongly encouraged to explore many different modeling methods (e.g. logistic regression, classification trees, SVC, etc.) but you should not include a detailed narrative of all of these attempts. At most this section should mention the methods explored and why they were rejected - most of your effort should go into describing the model you are using and your process for tuning and validatin it.*

*This section should also include the full implementation of your final model, including all necessary validation. As with figures, any included code must also be addressed in the text of the document.*

## 4. Discussion & Conclusions


*In this section you should provide a general overview of your final model, its performance, and reliability. You should discuss what the implications of your model are in terms of the included features, predictive performance, and anything else you think is relevant.*

*This should be written with a target audience of the client who is with the hotel data and university level mathematics but not necessarily someone who has taken a postgraduate statistical modeling course. Your goal should be to convince this audience that your model is both accurate and useful.*

*Keep in mind that a negative result, i.e. a model that does not work well predictively, that is well explained and justified in terms of why it failed will likely receive higher marks than a model with strong predictive performance but with poor or incorrect explinations / justifications.*

## 5. Convert Document

In [ ]:
# Run the following to render to PDF
!jupyter nbconvert --to pdf proj2.ipynb

[NbConvertApp] Converting notebook proj2.ipynb to pdf
[NbConvertApp] Writing 49741 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 72102 bytes to proj2.pdf


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d844b8b-c2b2-41b2-8d9c-38e9bb4a0873' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>